<h1 align="center">Hacettepe University<br>Department of Computer Science</h1> 
<h2 align="center">BBM 409 - Machine Learning Laboratory<br>2021 Fall</h2>
<h4 align="center">Assignment 2<br>Due on November 19, 2021</h4>



<table>
    <tr>
      <td><h4>Name Surname</h4></td>
      <td></td>
      <td><h4>Student ID</h4></td>
    </tr>
    <tr>
      <td><h4>Humeyra Uçar</h4></td>
      <td></td>
      <td><h4>21827957</h4></td>
    </tr>
    <tr>
      <td><h4>Murat Çelik</h4></td>
      <td></td>
      <td><h4>21827263</h4></td>
    </tr>
</table>


## 1. Introduction </a>

&emsp;&emsp; In this assignment, we implemented Decision Tree model by using ID3 algorithm on the Diabetes Risk Prediction dataset. A decision tree is a specific type of flow chart used to visualize the decision-making process by mapping out different courses of action.
In simple words, a decision tree is a structure that contains nodes (attributes) and branches and is built from a dataset. Each node is either used to make a decision (decision node) or represent an outcome (leaf node). <br>
&emsp;&emsp; We used the ID3 algorithm for create the tree. ID3 (Iterative Dichotomiser 3) is the algorithm that iteratively divides features into two or more groups at each step.

## 2. Background</a>
&emsp;&emsp;Before analyzing the model, we will conversation about which function we use to create the tree.

### ID3 Algorithm </a>
&emsp;&emsp; The ID3 algorithm selects the best feature at each step while building a Decision Tree. It uses Information Gain to find the best feature. Information gain measures how well a given attribute separates the training examples according to their target classification. The feature with the highest Information Gain is selected as the best one. We calculated information using the following formula:<br>
<center> $\normalsize Gain(S,A) = Entropy(S) - \sum_{v\epsilon\{Values(A)\}} (\frac{|S_v|}{S})Entropy(S_v) $ </center>
<br>
&emsp;&emsp;In order to define information gain precisely, we used a measure entropy.  Entropy characterizes the (im)purity of an arbitrary collection of examples.

&emsp;&emsp;You can see the entropy formula below:<br>
<center> $\normalsize Entropy(S) = - p_+ log_2p_+  - p_- log_2p_- $ </center>


### Performance Metrics </a>
&emsp;&emsp;Performance metrics help determine how well the model generalizes on new data.  Different performance metrics are used to evaluate different Machine Learning algorithms. For example, we used Accuracy, Precision, Recall, F1 score.

<b> Confusion Matrix</b><br>
&emsp;&emsp;A Confusion matrix is an NxN matrix used for evaluating the performance of a classification model, where N is the number of target classes. The matrix compares the actual target values with those predicted by the machine learning model. 
![image.png](attachment:image.png)
<b> Accuracy </b><br>  
&emsp;&emsp;Classification accuracy is the simplest metric to use and implement. It is calculated as you can see in the formula:
<center>  $ \normalsize Accuracy = 100 * \frac{num\;of\;correctly\;classified\;examples}{num\;of\;examples}$ </center>

<b> Precision </b><br>
&emsp;&emsp;Precision evaluates how precise a model is in predicting positive labels. Precision is a
good measure to determine when the cost of false positives is high.
<center> $\normalsize {Precision} = \frac{\mathrm{TP}}{\mathrm{TP} {+} \mathrm{FP}}$ </center>

<b> Recall </b><br>
&emsp;&emsp;Recall calculates the percentage of actual positives a model correctly identified. Recall-
When there is a high cost associated with false negatives.
<center> $ \normalsize {Recall} = \frac{\mathrm{TP}}{\mathrm{TP} {+}\mathrm{FN}}$ </center>

<b> F1 </b><br>
&emsp;&emsp;This is the harmonic mean of Precision and Recall and gives a better measure of the
incorrectly classified cases than the Accuracy Metric.
<center>  $ \normalsize F_{1}= 2 * \frac{\text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}$<center>
<br><br>    

<b> Comments about metrics: </b>
* Precision values show the accuracy of your positive predictions. If this value is high, it means that our error of telling people who are not sick that they are sick is low.
* Recall values show how much of the positive true values (sick people) we predicted correctly. The reason why this value is important compared to others is to say that you are not sick to a sick person, which can lead to fatal consequences. For this reason, our main goal should be to increase the accuracy of this value.
* Since the F1 score is calculated using precision and recall values, it is often used in problems where both values are important to us. For this data set, if we think that the wrong drugs given to people who are not sick will cause great harm (precision) and if we think that it will cause great harm if we miss the sick people (recall); We better use f1 score. 

### Cross-validation
![resim.png](attachment:2598cf1d-4843-4b63-83ae-33661f9b03d8.png)
<br>
```
1. Shuffle data
2. Divide the data into k different folds.
3. At each stage, equate one fold to the test data and the remaining data to the train data
4. Export the train and test data to the model and try the model
5. Save accuracy values 

```

In [1]:
def cross_val_score(model, X, y, label, pruning, k_split=5):   
    idx = X.index.values.copy()
    np.random.seed(42)   
    np.random.shuffle(idx)
    folds_idx = np.split(idx, k_split) 
    scores = pd.DataFrame()
    #scores = dict()
    score_accuracy = list()
    score_f1 = list()
    score_precision = list()
    score_recall = list()
    for i in range(k_split):
        tmp = folds_idx.copy()
        test_ind = tmp.pop(i)
        train_ind = [ _ for sub in tmp for _ in sub]
        X_train, X_test, y_train, y_test = X.iloc[train_ind], X.iloc[test_ind], y[train_ind], y[test_ind]
        model.fit(X_train, y_train, label,pruning)   # gives data to the model 
        y_pred = model.predict(X_test)               # output for test data by the model 
        score_f1.append(f1_score(y_test, y_pred, average="binary", pos_label="Positive"))
        score_precision.append(precision_score(y_test, y_pred, average="binary", pos_label="Positive"))
        score_recall.append(recall_score(y_test, y_pred, average="binary", pos_label="Positive"))
        score_accuracy.append(accuracy_score(y_test, y_pred))  
        
    scores["accuracy"] = score_accuracy
    scores["precision"] = score_precision
    scores["recall"] = score_recall
    scores["f1"] = score_f1
    scores = scores.T
    scores['min'] = scores.min(numeric_only=True, axis=1)
    scores['mean'] = scores.mean(numeric_only=True, axis=1)
    scores['max'] = scores.max(numeric_only=True, axis=1)
    return scores

## 3. Required Libraries <a name='3'></a>

In [2]:
import numpy as np 
import pandas as pd

    
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import matplotlib.pyplot as plt
import copy    

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score

## 4. Decision Tree Algorithms</a>

&emsp;&emsp; We use tree object for create decision tree. Tree object has two class variable as attribute and branches. 'Attribute' is the node of tree. And 'branches' is a dictionary which is values are tree object and keys are branches. <br>
&emsp;&emsp; DecisionTreeClassifier class has methods like fit, predict, pruning etc. The <i>fit</i> method takes the data and creates a tree. It then assigns root of the tree to the root variable of the object. If pruning is required, a tree is created by splitting the data into two (train/val). The <i>predict</i> method is used for the model to make a decision. By looking at the attributes of the given data, it moves through the branches until it reaches the leaf node.

```
   Tree creating algorithm:
1. For each attribute calculate information gain.
2. Select the attribute with the maximum information gain as the root node.
3. Split the dataset using the attribute that is root.
4. If all rows are in the same class(pure node), change the current root as a leaf node with the class.
5. Else repeat 1-4 step with rest of data and attributes until rows count equals 0 or there is no attribute to calculate information gain.
```


In [3]:
class DecisionTreeClassifier:
    """
    data : the data which using train the model 
    label : the class label 
    root : the root node of the model(decision tree)
    prune_list : hold pruning attribute
    """
    data = None 
    label = None
    root = None
    prune_list = None
    initial_root = False
    # The method we generate and optimize our tree for our model 
    def fit(self, X_train, y_train, label, pruning=False):    
        self.label = label
        self.root = Tree()
        self.prune_list = list()
        if pruning:
            # Train/Val/Test
            # We divide given train data by 2.(Train/Val)
            X_train_prune, X_dev, y_train_prune, y_dev = train_test_split(X_train, y_train, test_size=0.25, random_state=42) 
            self.data = X_train_prune.copy()
            self.data[label] = y_train_prune.copy()
            Tree.create_tree(self.root, None, self.data, self.label, list(X_train.columns))
            Tree.stablize_tree(self.root, self.data, self.label)
            if self.initial_root:
                self.draw()
            self.pruning(self.root, self.data, self.label, X_dev, y_dev)      
        else:
            self.data = X_train.copy()
            self.data[label] = y_train.copy()
            Tree.create_tree(self.root, None, self.data, self.label, list(X_train.columns))
            Tree.stablize_tree(self.root, self.data, self.label)
            
    # The method which the model decides.
    def predict(self, X_test):
        y_pred = list()
        for x in range(len(X_test)):
            row = X_test.iloc[x]
            att = row[self.root.attribute]
            node = self.root.branches[att]
            while isinstance(node, Tree):
                att = row[node.attribute]
                node = node.branches[att]
            y_pred.append(node)
        return y_pred
    
    def draw(self):
        self.root.draw_tree()
    def change_initial_root(self):
        self.initial_root = True
    def pruning(self, node, data, label, X_dev, y_dev):
        prune_model = copy.deepcopy(self)        
        twig = dict()
        self.minGain(node, data, label, twig) # Catalog all twigs in the tree 
        attribute = min(twig, key = twig.get) # Find the twig with the least Information Gain
        self.cut(prune_model.root, attribute, data, label) # Relabel twig as a leaf
       
        accuracy_prune = prune_model.check_accuracy(X_dev, y_dev)
        accuracy_prev =  self.check_accuracy(X_dev, y_dev) # Measure the accuracy value of your decision tree
        if accuracy_prune >= accuracy_prev:   # If ”Current Accuracy ≥ Last Accuracy” : Jump to ”pruning”
            self.prune_list.append(str("Prune attribute : "+ attribute + " , {Score acc_prune : "+ str(accuracy_prune)+", Score acc_prev : " + str(accuracy_prev)+ "\n"))
            self.root = prune_model.root
            self.pruning(self.root, data, label, X_dev, y_dev)
            
    def check_accuracy(self, X_dev, y_dev):
        y_pred = self.predict(X_dev)
        accuracy_prune = accuracy_score(y_pred, y_dev)
        return accuracy_prune
                
    def cut(self, node, attribute, data, label):
        for k, v in node.branches.items():
            if isinstance(v, str):
                continue
            if v.attribute == attribute:
                new_label= data[data[node.attribute] == k][label].value_counts().index[0]
                node.branches[k] = new_label
            else:
                next_node = v
                feature_value_data = data[data[node.attribute] == k]
                self.cut(next_node, attribute, feature_value_data, label)
                
    def minGain(self, root_node, data, label, twig):
        if False not in list(map(lambda x : isinstance(x, str), root_node.branches.values())):
            twig[root_node.attribute] = Tree.gain(data, root_node.attribute, label)
        else:
            for k, v in root_node.branches.items():
                next_node = v
                if not isinstance(v, str):
                    feature_value_data = data[data[root_node.attribute] == k]
                    self.minGain(next_node, feature_value_data, label, twig)
                    

In [4]:
class Tree:
    """
        attribute : name of Node (columns of data)
        branches : list of Edge (labels in columns)
    """
    attribute = None
    branches = None
    
    def __init__(self, attribute=None):
        self.attribute = attribute
        self.branches = dict()
        
    def add(self, attribute, subtree):
        self.branches[attribute] = subtree

    def __repr__(self):
        return '"Node" : "' + str(self.attribute) + '","Branches":' + str(self.branches)+''
    
    # draw tree
    def print_tree(self):
        G = nx.DiGraph()
        G.add_node(self.attribute)
        for branch, node in list(self.branches.items()): 
            if isinstance(node, Tree):
                G.add_edge(self.attribute, node.attribute, label=branch)
                H = node.print_tree()
                G = nx.compose(G, H)
            else:
                G.add_edge(self.attribute, node+str("_"+self.attribute[:5]), label=branch)
        return G
    
    def draw_tree(self):
        fig = plt.figure(1, figsize=(20, 20), dpi=100)
        G = self.print_tree()
        pos = graphviz_layout(G, prog="dot")
        edge_labels = nx.get_edge_attributes(G, 'label')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        nx.draw(G, pos, with_labels=True, font_size=15)
        plt.show()
    
    # Entropy and Gain Methods 
    @staticmethod
    def max_gain(data, column, label):
        arr = dict()
        for x in column:
            arr[x] = Tree.gain(data, x, label)
        return max(arr, key=arr.get)

    @staticmethod
    def entropyS(data, label):
        p = data[label].value_counts() / data.shape[0]
        return np.sum(-p * np.log2(p))
    
    @staticmethod
    def entropy(data, attribute, v, label):
        sv_ratio = data[data[attribute] == v][label].value_counts() / data[data[attribute] == v][label].shape[0]
        entropy_sv = np.sum(-sv_ratio * np.log2(sv_ratio))
        return entropy_sv
    
    @staticmethod
    def gain(data, attribute, label):
        gain = Tree.entropyS(data, label)    
        ratio_sv_s = data[attribute].value_counts() / data[attribute].shape[0]
        for v in data[attribute].unique():
            entropy_sv = Tree.entropy(data, attribute, v, label)
            gain -= ratio_sv_s[v] * entropy_sv
        return gain
    
    # Our main method creating the model
    @staticmethod
    def create_tree(root, prev_feature_value, train_data, label,columns):
        if train_data.shape[0] != 0 and len(columns)>0: 
            
            subtree = "invalid"
            while subtree == "invalid":
                if len(columns)<=0:
                    return
                max_info_feature = Tree.max_gain(train_data, columns,label ) # we find the node to add to our tree 
                columns.remove(max_info_feature)  # we delete the used node from our list 
                # we create our subtree consisting of single node and edge 
                subtree, train_data = Tree.generate_sub_tree(max_info_feature, train_data, label)
            next_root = None

            if prev_feature_value != None:  # we add subtree to our tree 
                new_node = subtree
                root.add(prev_feature_value, new_node)
                next_root = new_node
            else:   # we assign the node with the highest gain to our root node. 
                root.attribute = subtree.attribute
                root.branches = subtree.branches
                next_root = subtree

            for branch, node in list(next_root.branches.items()): # fill the edge of our tree 
                if node == "?": 
                    feature_value_data = train_data[train_data[max_info_feature] == branch] 
                    Tree.create_tree(next_root, branch, feature_value_data, label,columns) 
 
    @staticmethod
    def generate_sub_tree(feature_name, train_data, label):
        subtree = Tree(feature_name) # create a subtree with column with max gain 
        feature_value_count_dict = dict(train_data[feature_name].value_counts())
        if len(feature_value_count_dict)< 2 :  # If our node has only one child remove it 
            return "invalid",train_data
        for feature_value, count in feature_value_count_dict.items():
            feature_value_data = train_data[train_data[feature_name] == feature_value] 

            leaf_node_control = False
            for c in train_data[label].value_counts().keys(): #for each output class
                class_count = feature_value_data[feature_value_data[label] == c].shape[0] 

                # if node data is completely from a particular class then assign class to that node 
                if class_count == count: 
                    subtree.add(feature_value, c)     
                    train_data = train_data[train_data[feature_name] != feature_value] 
                    leaf_node_control = True
            # if the node data is not entirely of a particular class, assign "?" to continue. 
            if not leaf_node_control: 
                    subtree.add(feature_value, "?")      
        return subtree, train_data 
    
    @staticmethod
    def stablize_tree(root,data,label):
        # If data is empty, then below this new branch add a leaf node with label = most common value of data
        for k,v in root.branches.items():
            if isinstance(v, str):
                if v == "?":
                    root.branches[k] = data[label].value_counts().index[0]
                else :
                    continue
            else:
                Tree.stablize_tree(v, data[data[root.attribute] == k], label)
            

## 5. Part I: Diabetes Risk Prediction </a>

### 5.1 Data Analysis </a>
&emsp;&emsp; Diabetes dataset consists of 520 samples with 16 features two class types. Classes are "Positive" and "Negative". We have 16 attributes, including the continuous "Age" attribute. In order to use our features in training, we make discretization to the "age" feature.
<br>
&emsp;&emsp; Discretization transforms the continuous variables in our data into discrete data reduced to certain intervals. 

Attribute and Class Information:
1. Age (Continuous Attribute)
2. Gender (Discrete Attribute)
3. Polyuria (Discrete Attribute)
4. Polydipsia (Discrete Attribute)
5. Sudden Weight Loss (Discrete Attribute)
6. Weakness (Discrete Attribute)
7. Polyphagia (Discrete Attribute)
8. Genital Thrush (Discrete Attribute)
9. Visual Blurring (Discrete Attribute)
10. Itching (Discrete Attribute)
11. Irritability (Discrete Attribute)
12. Delayed Healing (Discrete Attribute)
13. Partial Paresis (Discrete Attribute)
14. Muscle Stiffness (Discrete Attribute)
15. Alopecia (Discrete Attribute)
16. Obesity (Discrete Attribute)
17. Class (Output Prediction Class Information, ”Positive” or ”Negative”)

In [5]:
diabetes = pd.read_csv('diabetes_data_upload.csv')
diabetes['Age']=pd.cut(diabetes['Age'], 5, labels=['15-30', '31-45', '46-60', '61-75', '76-90']) #discretization
diabetes.head()

FileNotFoundError: ignored

### 5.2 Model Analysis <a name='5.2'></a>

In [ ]:
label = "class"
X = diabetes.drop([label], axis=1)
y = diabetes[label]

In [ ]:
model = DecisionTreeClassifier()
df_part1 = cross_val_score(model, X, y, label, False)
df_part1

&emsp;&emsp; Our lowest value in Precision value is 89%. It seems that we have achieved an precision of 100%. This indicates the low rate of predict "sick" to who are not sick. <br>
&emsp;&emsp; In our opinion, Recall value is the most important among these metrics. The efficiency of this value is important for us to be one step ahead of the disease. We must increase the efficiency of this value.<br>
&emsp;&emsp; We think that our model could be better with a minimum value of 80% in our F1 score, but it is still positive for the worst scenario. 

## 6. Part II: Pruning Decision Tree</a>
#### Post-pruning
&emsp; &emsp; Pruninig is a technique applied on the tree so that decision trees can be run more efficiently and more accurately. Post-pruning, first build the decision tree and then remove any non-essential branches. You can see our algorithm below: 

```
   Pruning Algorithm:
1. Catalog all twigs in the tree
2. Find the twig with the least Information Gain
3. Remove all child nodes of the twig
4. Relabel twig as a leaf (Set the majority of ”Positive” or ”Negative”
as leaf value)
5. Measure the accuracy value of your decision tree model with removed
twig on the validation set (”Current Accuracy”)
6. If ”Current Accuracy ≥Last Accuracy” : Jump to ”Step1”
7. Else : Revert the last changes done in Step 3,4 and then terminate
```

In [ ]:
model = DecisionTreeClassifier()
df_part2 = cross_val_score(model,X,y,"class",True)
df_part2

#### Pre-pruning
&emsp;&emsp; One of the ways to avoid overfitting is to check for cross-validation error at each stage of splitting the tree, if the error does not decrease significantly enough, we stop the tree-building process early before we produce leaves with very small samples. The disadvantage of pre-pruning is may may underfit by stopping too early.



## 7. Error Analysis for Classification</a>

&emsp;&emsp; We compare the error calculations of our pre-pruning model with our post-pruning model. 

In [ ]:
df_part1[["min","mean","max"]].join(df_part2[["min","mean","max"]],lsuffix="_pre-prune",rsuffix="_post-prune")

&emsp;&emsp; When we look at the table, we see that our values increase with pruning. Pruning gives us both accuracy and complexity. 

In [ ]:
df_part1[["mean"]].join(df_part2[["mean"]],lsuffix="_pre-prune",rsuffix="_post-prune")

#### Missclassified Examples
1. When we apply the discretization method to the age column, we make some transformations. In our data, only people with different age characteristics seem to be the same age with this technique. This creates a conflict in the data when the classes are different. This increases the error rate.

2. When building our tree, at some leaf nodes we assign the most common value to that leaf because of the algorithm in data of that part. Most non-common data here increases the error.

3. Although many rows in the training data has exactly same features, they are labeled differently. It will causes missclasify and it decrease accuracy.

In [ ]:
same_rows = [315, 321, 357, 89, 162, 267, 364, 455]
diabetes.iloc[same_rows]

## 8. Time </a>

In [ ]:
import time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

&emsp;&emsp;For our Decision tree model, we create our tree once with the fit method. This is the longest part of the algorithm. The Predict method operates on our tree that we created. For this reason, the duration of the fit method is greater than the duration of the predict method. 

In [ ]:
model = DecisionTreeClassifier()
start = time.time()
model.fit(X_train, y_train, "class",False)   
after_fit = time.time()
y_pred = model.predict(X_test)
after_predict = time.time()
f1_score(y_test,y_pred,average="binary",pos_label="Positive")
fit_time = after_fit-start
predict_time = after_predict - after_fit
print("Fitting Time : ", fit_time)
print("Predict Time : ", predict_time)

&emsp;&emsp;The model we pruning is slower than the model we don't pruning because we do additional processing. We observe this with the following time. 

In [ ]:
model = DecisionTreeClassifier()
start = time.time()
model.fit(X_train, y_train, "class",True)  
after_fit = time.time()
y_pred = model.predict(X_test)
after_predict = time.time()
f1_score(y_test,y_pred,average="binary",pos_label="Positive")
fit_time = after_fit-start
predict_time = after_predict - after_fit
print("Fitting Time : ", fit_time)
print("Predict Time : ", predict_time)

## Optional

In [ ]:
import pydot

#### Pre-pruning Decision Tree

In [ ]:
model = DecisionTreeClassifier()
model.change_initial_root()
model.fit(X_train, y_train, "class",True)   

#### Post-pruning Decision Tree

In [ ]:
for x in model.prune_list:
    print(x)
model.draw()

## References
- Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems by Aurélien Géron
- [towardsdatascience](https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761)
- [Stanford CS229: Machine Learning](https://www.youtube.com/watch?v=jGwO_UgTS7I&list=PLoROMvodv4rMiGQp3WXShtMGgzqpfVfbU)